In [1]:
from __future__ import print_function, division
import numpy as np
np.random.seed(42)
import tensorflow as tf
import pdb
import fxns

/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# data locations
raw_data_path = '../data/shuffled_examples'
outbase = '../output/shuffled_examples'
train_out_path = '%s/texttrain' % outbase
valid_out_path = '%s/textvalidate' % outbase
test_out_path = '%s/texttest' % outbase

# training hyperparameters
batch_size = 50
num_batches_in_train = int(434786 / batch_size)
num_epochs = 10
capacity = 2000
min_after_dequeue = 1000
learning_rate = 0.01

# retrieving models
global_step_to_load = None

In [3]:
tf.reset_default_graph()

# get training data
(seq, label), info = fxns.get_seq_and_label(train_out_path)
seq_batch, label_batch = tf.train.shuffle_batch([seq, label],
    batch_size=batch_size,
    capacity=capacity,
    min_after_dequeue=min_after_dequeue)
print('inputs are of size', info['seq_len'])

# get validation data
(seqv, labelv), infov = fxns.get_seq_and_label(valid_out_path)
seqv_batch, labelv_batch = tf.train.shuffle_batch([seqv, labelv],
    batch_size=batch_size,
    capacity=capacity,
    min_after_dequeue=min_after_dequeue)
print('outputs are of size', info['label_len'])

inputs are of size 4000
outputs are of size 24


/anaconda2/lib/python2.7/site-packages/pandas/core/series.py:2890: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  infer_datetime_format=infer_datetime_format)


In [4]:
# # get model -- logistic
# import logreg_model
# modelname='logreg'
# X, Y, loss, logits = logreg_model.get_logreg_model(info['seq_len'], info['label_len'])

# get model -- generic convnet
import novel_model; reload(novel_model)
modelname='freeconv'
#conv_infos = [(7,(1,20),(2,2),4),(5,(1,20),(2,2),7)]#, (8,(1,20),(2,2),5)]
conv_infos = [(14,(1,21),(2,2),2,4),(14,(1,8),(2,2),2,14),(7,(1,5),(2,2),2,14)]#, (8,(1,20),(2,2),5)]
X, Y, loss, logits = novel_model.get_novel_model(info['seq_len'], info['label_len'], conv_infos)

500.0
250.0
125.0
4000 h 15 125.0


In [ ]:
# define optimizer
with tf.name_scope('optimizer'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate,
                                                 name='SGD-Optimizer')
    update = optimizer.minimize(loss)

# define other summaries we want
with tf.name_scope('summaries'):
    tf.summary.scalar('loss', loss)
    tf.summary.histogram('histogram-loss', loss)
    summary_op = tf.summary.merge_all()

In [ ]:
if tf.gfile.Exists('log/' + modelname):
   tf.gfile.DeleteRecursively('log/' + modelname) 

# train

print_every = 25
save_every = 250

with tf.Session() as sess:
    
    saver = tf.train.Saver()
    
    if global_step_to_load is None:
        global_step = 0
    else:
        saver.restore(sess, 'log/%s-%d' % (modelname, global_step_to_load))
        global_step = global_step_to_load
    
#    print(layer)
    
    writer = tf.summary.FileWriter('log/' + modelname + '/train', sess.graph)
    writerv = tf.summary.FileWriter('log/' + modelname + '/valid')
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    if global_step_to_load is None:
        sess.run(tf.global_variables_initializer())
    
    for epoch in range(num_epochs):
        total_loss = 0
        for i in range(num_batches_in_train):
            batch_X, batch_Y = sess.run([seq_batch, label_batch])
            batch_Xv, batch_Yv = sess.run([seqv_batch, labelv_batch])
            _, _loss = sess.run([update, loss],
                            feed_dict={X: batch_X, Y: batch_Y})
            total_loss += _loss #/ num_batches_in_train
            
            summary = sess.run(summary_op, feed_dict={X: batch_X,
                                                    Y: batch_Y})
            summaryv = sess.run(summary_op, feed_dict={X: batch_Xv,
                                                    Y: batch_Yv})
#            writer.add_summary(summary, global_step=epoch*num_batches_in_train + i)
#            writerv.add_summary(summaryv, global_step=epoch*num_batches_in_train + i)
            writer.add_summary(summary, global_step=global_step)
            writerv.add_summary(summaryv, global_step=global_step)
            
            global_step += 1
            
            if global_step % save_every == 0:
                print('saving')
                print('Epoch:',epoch,'global_step',global_step,'i',i,'Avg loss in epoch(incomplete):',total_loss / (i+1))
                saver.save(sess, 'log/%s' % modelname, global_step=global_step)
                
            if global_step % print_every == 0:
                print('Epoch:',epoch,'global_step',global_step,'i',i,'Avg loss in epoch(incomplete):',total_loss / (i+1))
        
        print('saving')
        print('Epoch:',epoch,'global_step',global_step,'i',i,'Avg loss in epoch(incomplete):',total_loss / (i+1))

        saver.save(sess, 'log/%s' % modelname, global_step=global_step)
    
    writer.close()
    coord.request_stop()
    coord.join(threads)

Epoch: 0 global_step 25 i 24 Avg loss in epoch(incomplete): 0.4555277693271637
Epoch: 0 global_step 50 i 49 Avg loss in epoch(incomplete): 0.33252002477645876
Epoch: 0 global_step 75 i 74 Avg loss in epoch(incomplete): 0.2789812296628952
Epoch: 0 global_step 100 i 99 Avg loss in epoch(incomplete): 0.25089493215084074
Epoch: 0 global_step 125 i 124 Avg loss in epoch(incomplete): 0.23400295746326447
Epoch: 0 global_step 150 i 149 Avg loss in epoch(incomplete): 0.22268974483013154
Epoch: 0 global_step 175 i 174 Avg loss in epoch(incomplete): 0.21443751573562622
Epoch: 0 global_step 200 i 199 Avg loss in epoch(incomplete): 0.20836703836917878
Epoch: 0 global_step 225 i 224 Avg loss in epoch(incomplete): 0.2035365508000056
saving
Epoch: 0 global_step 250 i 249 Avg loss in epoch(incomplete): 0.19972987306118012
Epoch: 0 global_step 250 i 249 Avg loss in epoch(incomplete): 0.19972987306118012
Epoch: 0 global_step 275 i 274 Avg loss in epoch(incomplete): 0.19665270512754268
Epoch: 0 global_ste

Epoch: 0 global_step 2275 i 2274 Avg loss in epoch(incomplete): 0.16888985075793422
Epoch: 0 global_step 2300 i 2299 Avg loss in epoch(incomplete): 0.16885713125700536
Epoch: 0 global_step 2325 i 2324 Avg loss in epoch(incomplete): 0.16881459568777393
Epoch: 0 global_step 2350 i 2349 Avg loss in epoch(incomplete): 0.16877209795282241
Epoch: 0 global_step 2375 i 2374 Avg loss in epoch(incomplete): 0.1687316463370072
Epoch: 0 global_step 2400 i 2399 Avg loss in epoch(incomplete): 0.1687027858446042
Epoch: 0 global_step 2425 i 2424 Avg loss in epoch(incomplete): 0.1686640146098186
Epoch: 0 global_step 2450 i 2449 Avg loss in epoch(incomplete): 0.1686280252069843
Epoch: 0 global_step 2475 i 2474 Avg loss in epoch(incomplete): 0.16859648368575356
saving
Epoch: 0 global_step 2500 i 2499 Avg loss in epoch(incomplete): 0.16856838999986648
Epoch: 0 global_step 2500 i 2499 Avg loss in epoch(incomplete): 0.16856838999986648
Epoch: 0 global_step 2525 i 2524 Avg loss in epoch(incomplete): 0.1685291

Epoch: 0 global_step 4525 i 4524 Avg loss in epoch(incomplete): 0.16703673733532098
Epoch: 0 global_step 4550 i 4549 Avg loss in epoch(incomplete): 0.16702528297901154
Epoch: 0 global_step 4575 i 4574 Avg loss in epoch(incomplete): 0.16701887124548845
Epoch: 0 global_step 4600 i 4599 Avg loss in epoch(incomplete): 0.16701051518968915
Epoch: 0 global_step 4625 i 4624 Avg loss in epoch(incomplete): 0.16699829885444126
Epoch: 0 global_step 4650 i 4649 Avg loss in epoch(incomplete): 0.16698510623747304
Epoch: 0 global_step 4675 i 4674 Avg loss in epoch(incomplete): 0.1669716313242275
Epoch: 0 global_step 4700 i 4699 Avg loss in epoch(incomplete): 0.16696330473144003
Epoch: 0 global_step 4725 i 4724 Avg loss in epoch(incomplete): 0.1669522609944066
saving
Epoch: 0 global_step 4750 i 4749 Avg loss in epoch(incomplete): 0.16694336576524535
Epoch: 0 global_step 4750 i 4749 Avg loss in epoch(incomplete): 0.16694336576524535
Epoch: 0 global_step 4775 i 4774 Avg loss in epoch(incomplete): 0.16693

Epoch: 0 global_step 6775 i 6774 Avg loss in epoch(incomplete): 0.16638310879798832
Epoch: 0 global_step 6800 i 6799 Avg loss in epoch(incomplete): 0.16637879041187903
Epoch: 0 global_step 6825 i 6824 Avg loss in epoch(incomplete): 0.16637767775154813
Epoch: 0 global_step 6850 i 6849 Avg loss in epoch(incomplete): 0.1663731400026892
Epoch: 0 global_step 6875 i 6874 Avg loss in epoch(incomplete): 0.16636250557249244
Epoch: 0 global_step 6900 i 6899 Avg loss in epoch(incomplete): 0.16635639325432156
Epoch: 0 global_step 6925 i 6924 Avg loss in epoch(incomplete): 0.1663503678438896
Epoch: 0 global_step 6950 i 6949 Avg loss in epoch(incomplete): 0.16634522421754522
Epoch: 0 global_step 6975 i 6974 Avg loss in epoch(incomplete): 0.16634205773739832
saving
Epoch: 0 global_step 7000 i 6999 Avg loss in epoch(incomplete): 0.16633602322212288
Epoch: 0 global_step 7000 i 6999 Avg loss in epoch(incomplete): 0.16633602322212288
Epoch: 0 global_step 7025 i 7024 Avg loss in epoch(incomplete): 0.16632

saving
Epoch: 1 global_step 9000 i 304 Avg loss in epoch(incomplete): 0.16490839597631674
Epoch: 1 global_step 9000 i 304 Avg loss in epoch(incomplete): 0.16490839597631674
Epoch: 1 global_step 9025 i 329 Avg loss in epoch(incomplete): 0.16489045895410306
Epoch: 1 global_step 9050 i 354 Avg loss in epoch(incomplete): 0.1648878014843229
Epoch: 1 global_step 9075 i 379 Avg loss in epoch(incomplete): 0.16489398004977326
Epoch: 1 global_step 9100 i 404 Avg loss in epoch(incomplete): 0.16488549337710864
Epoch: 1 global_step 9125 i 429 Avg loss in epoch(incomplete): 0.16489418024240537
Epoch: 1 global_step 9150 i 454 Avg loss in epoch(incomplete): 0.16488261615837013
Epoch: 1 global_step 9175 i 479 Avg loss in epoch(incomplete): 0.16495385880892474
Epoch: 1 global_step 9200 i 504 Avg loss in epoch(incomplete): 0.1649645954075426
Epoch: 1 global_step 9225 i 529 Avg loss in epoch(incomplete): 0.1649880033337845
saving
Epoch: 1 global_step 9250 i 554 Avg loss in epoch(incomplete): 0.16497693252

Epoch: 1 global_step 11200 i 2504 Avg loss in epoch(incomplete): 0.16501445487706723
Epoch: 1 global_step 11225 i 2529 Avg loss in epoch(incomplete): 0.1650170661185099
saving
Epoch: 1 global_step 11250 i 2554 Avg loss in epoch(incomplete): 0.16501362928555668
Epoch: 1 global_step 11250 i 2554 Avg loss in epoch(incomplete): 0.16501362928555668
Epoch: 1 global_step 11275 i 2579 Avg loss in epoch(incomplete): 0.16501448911636374
Epoch: 1 global_step 11300 i 2604 Avg loss in epoch(incomplete): 0.16501045375776383
Epoch: 1 global_step 11325 i 2629 Avg loss in epoch(incomplete): 0.16501539773360857
Epoch: 1 global_step 11350 i 2654 Avg loss in epoch(incomplete): 0.16502244022943205
Epoch: 1 global_step 11375 i 2679 Avg loss in epoch(incomplete): 0.16503372137337477
Epoch: 1 global_step 11400 i 2704 Avg loss in epoch(incomplete): 0.1650333434122988
Epoch: 1 global_step 11425 i 2729 Avg loss in epoch(incomplete): 0.16504125969532207
Epoch: 1 global_step 11450 i 2754 Avg loss in epoch(incomple

Epoch: 1 global_step 13375 i 4679 Avg loss in epoch(incomplete): 0.16505373880330823
Epoch: 1 global_step 13400 i 4704 Avg loss in epoch(incomplete): 0.1650517250855595
Epoch: 1 global_step 13425 i 4729 Avg loss in epoch(incomplete): 0.16505058107240003
Epoch: 1 global_step 13450 i 4754 Avg loss in epoch(incomplete): 0.16505191878314776
Epoch: 1 global_step 13475 i 4779 Avg loss in epoch(incomplete): 0.16504927843684433
saving
Epoch: 1 global_step 13500 i 4804 Avg loss in epoch(incomplete): 0.16504900062865696
Epoch: 1 global_step 13500 i 4804 Avg loss in epoch(incomplete): 0.16504900062865696
Epoch: 1 global_step 13525 i 4829 Avg loss in epoch(incomplete): 0.16504768852614962
Epoch: 1 global_step 13550 i 4854 Avg loss in epoch(incomplete): 0.16505155555067544
Epoch: 1 global_step 13575 i 4879 Avg loss in epoch(incomplete): 0.1650588305361691
Epoch: 1 global_step 13600 i 4904 Avg loss in epoch(incomplete): 0.16506337573282823
Epoch: 1 global_step 13625 i 4929 Avg loss in epoch(incomple